In [1]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload

import xarray as xr
import numpy as np
import os
import pandas as pd
from glob import glob
import datetime as dt
from PIL import Image
import sys
import functions as f
import pickle
import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras import backend as K
from tensorflow.keras.utils import plot_model
from IPython.display import Image 
from tensorflow.python.data.ops import dataset_ops
import dask
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from io import StringIO
import shap
from keras.layers import Input
import keras as k
from contextlib import redirect_stdout
from PIL import Image
import matplotlib.gridspec as gridspec
import cv2
import channelExperiment as CE
import loadDataWeek0 as loadData
import bottleneck as bn
from tensorflow.keras import mixed_precision
import csv
import addPredictors as pred
import loadValues as lv
import masks
import denseValue


global region_name,testing_scenario,save_loss_name, final_testing_year
region_name = 'CONUS' #['australia','CONUS', 'china']

#This is only for CONUS so far. This allows us to do a semi-cross validation experiment
final_testing_year = 2012 # [None,2012]   ### options. None = 2018-2019 testing year, 2012 = 2011-2012 testing years

if region_name != 'CONUS':
    assert final_testing_year == None, 'If region is not CONUS, you must make final_testing_year == None'

global ref_source
ref_source = 'GEFSv12'

#Tensorflow RT things
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
tf.config.optimizer.set_jit(True) # Enable XLA.


global RZSM_or_Tmax_or_both
RZSM_or_Tmax_or_both = 'RZSM' # for getting the predictor from either RZSM and Tmax ('both') or only RZSM ('RZSM')

global num_predictions_testing
num_predictions_testing = 25

global num_train_val_predictions
num_train_val_predictions=10

#for permutation plot
max_RZSM_value,max_tmax_value = 0.05,5

#set for the larger memeory ones
# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'




2024-05-21 15:16:47.628956: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 15:16:48.366609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/glade/work/klesinger/conda-envs/tf212gpu_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100-SXM4-80GB, compute capability 8.0


# Choose different loss and architecture configurations

In [2]:

#First just re-save the new density value into a new file (it's convoluted but it works)
# denseValue.return_dense_value_file(region_name, 0.5) #can chooose 0.5 or 0.8 as the alpha coefficient


import losses

testing_scenario = 'regular' #['dense', 'regular', 'transformer', 'super_pixel', 'attention', 'denseLarge', 'basic']

global save_loss_name, loss_fn

if testing_scenario == 'dense':
    import modelRzsmDenseRelu as UNETRzsm
    
    save_loss_name = 'denseLoss'
    loss_fn = losses.crps2d_tf_dense_test2
    
elif testing_scenario == 'regular':
    import modelRzsmReluExtraConv as UNETRzsm

    save_loss_name = 'regular'
    loss_fn = losses.crps2d_tf

elif testing_scenario == 'transformer':
    import modelRzsmReluTransformer as UNETRzsm
    
    save_loss_name = 'transformer'
    loss_fn = losses.crps2d_tf_dense_test2

elif testing_scenario == 'super_pixel':
    import modelRzsmReluPixelSuper as UNETRzsm
    
    save_loss_name = 'supPixel'
    loss_fn = losses.crps2d_tf_dense_test2

elif testing_scenario == 'attention':
    import modelRzsmReluAttention as UNETRzsm
    
    save_loss_name = 'attention'
    loss_fn = losses.crps2d_tf_dense_test2
    
elif testing_scenario == 'None':
    import modelRzsm2DdropoutRelu as UNETRzsm
    
    save_loss_name = ''
    loss_fn = losses.crps2d_tf

elif testing_scenario == 'denseLarge':
    import modelRzsmReluExtraConv as UNETRzsm

    save_loss_name = 'denseLarge'
    loss_fn = losses.crps2d_tf_dense_test2
    
elif testing_scenario == 'basic':
    import basicUNET as UNETRzsm

    save_loss_name = testing_scenario
    loss_fn = losses.crps2d_tf

2024-05-21 15:17:20.246303: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79084 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:01:00.0, compute capability: 8.0


Using TensorFlow backend


In [3]:
# #check loss values
# def open_loss(experiment):
#     loss = pd.read_csv(f'Losses_with_OBS/Wk_1/Wk1_{experiment}')
#     print(loss)


In [4]:
# open_loss('EX12')

In [5]:
# # #Load model and check summary
# model = load_model(f'checkpoints/Wk_1/Wk1_EX5',compile=False) #don't need the custom loss function for predictions
# model.summary() 


# This script will load data based on specific lead times that we want to experiment with and what Experiments we choose to run

# Week 1 (lead index 6 of the forecast)

| Ex. Number/Name | Obs. RZSM Inputs | Obs. Other Inputs                    | UNET predicted Inputs (N>=2)     |Reforecast Inputs| Prediction Lead Week/Variables | Activation function | Loss function | Batch size |
| ----------------| -----------------| ------------------------------------ | -------------------      |--------          | -------------------| ---------------------| ------------------------| ------------|
| 0 - EX0         | None             | None                                 | None                     | RZSM, Wk 1-N| Wk N     -  RZSM |              Relu             | CRPS experimental| 66
| 1 - EX1         | Wk. lags 1-3    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | RZSM Wk 1-N |        None |  Wk N     -  RZSM |             Relu             | CRPS experimental | 66
| 2 - EX2         | Wk. lags 1-6   | pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N          |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 3 - EX3         | Wk. lags 1-9    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | RZSM Wk 1-N         |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 4 - EX4         | Wk. lags 1-12    | pwat,spfh,tmax,z200,diff_temp lags 1-3| RZSM Wk 1-N         |  None |  Wk N     -  RZSM |      Relu             | CRPS experimental| 66
| 5 - EX5         | Wk. lags 1-3   | None                                   | None                   | RZSM, week N | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 6 - EX6         | Wk. lags 1-6   | None                                     | None                   |RZSM, week N  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 7 - EX7         | Wk. lags 1-9    | None                                | None                   |RZSM, week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 8 - EX8         | Wk. lags 1-12    | None                                  | None                   |RZSM, week N  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 9 - EX9         | Wk. lags 1-3    |pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 10 - EX10       | Wk. lags 1-6    |pwat,spfh,tmax,z200,diff_temp  lags 1-3 | RZSM Wk 1-N                  |RZSM  week N  |Wk N     -  RZSM   |         Relu             | CRPS experimental| 66
| 11 - EX11       | Wk. lags 1-9    |pwat,spfh,tmax,z200,diff_temp  lags 1-3 | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 12 - EX12       | Wk. lags 1-12   |pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 13 - EX13       | None             | None                                 | None                     | RZSM week N| Wk N     -  RZSM |              Relu             | CRPS experimental| 66
| 14 - EX14         | Wk. lags 1-3    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | None |        None |  Wk N     -  RZSM |             Relu             | CRPS experimental | 66
| 15 - EX15         | Wk. lags 1-6   | pwat,spfh,tmax,z200,diff_temp lags 1-3  | None             |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 16 - EX16         | Wk. lags 1-9    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | None            |  None|  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 17 - EX17        | Wk. lags 1-12    | pwat,spfh,tmax,z200,diff_temp lags 1-3| None            |  None|  Wk N     -  RZSM |      Relu             | CRPS experimental| 66
| 18 - EX18         | Wk. lags 1-3   | None                                   | RZSM Wk 1-N                 | RZSM, week N | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 19 - EX19         | Wk. lags 1-6   | None                                     | RZSM Wk 1-N                  |RZSM, week N  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 20 - EX20         | Wk. lags 1-9    | None                                | RZSM Wk 1-N                  |RZSM, week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 21 - EX21         | Wk. lags 1-12    | None                                  | RZSM Wk 1-N                   |RZSM, week N  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 22 - EX22         | Wk. lags 1-3   | None                                   | None                   | None | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 23 - EX23         | Wk. lags 1-6   | None                                     | None                   |None  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 24 - EX24         | Wk. lags 1-9    | None                                | None                   |None  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 25 - EX25         | Wk. lags 1-12    | None                                  | None                   |None  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 26 - EX26         | None    | None                                  | None                   |Wk 1-N (choose best models)  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 27 - EX27         | Wk. lags 1-3    |pwat,spfh,tmax,z200,diff_temp lags 1-3  |  None                   |Wk1 RZSM,tmax, diff_temp, z200, pwat, spfh  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66 (like EX9)
| 28 - EX28         | Wk. lags 1-6    |pwat,spfh,tmax,z200,diff_temp lags 1-3  |  None                   |Wk1 RZSM,tmax, diff_temp, z200, pwat, spfh  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66 (like EX10)

# EX 1-4 - Testing if the prediction of previous week adds value (but don't include the current week that is trying to be predicted). Observation driven but with a prediction.
# EX 14-17 - Testing if the prediction of previous week adds value (but don't include the current week that is trying to be predicted). Purely observation driven.
# EX 5-8 - Testing if adding other observations (pwat, spfh, etc) has an increase or decrease in skill.
# EX 18-21 - Seeing if adding the previous week gains additional skill
# EX 22-25 - Purely observation driven. Only soil moisture.

In [6]:
input_directory = f'Data/model_npy_inputs/{region_name}/Model_input_data' #For inputs into the model
verification_directory = f'Data/model_npy_inputs/{region_name}/Verification_data' #For observation verification
start_of_forecast_init = '2000-01-05'
end_of_forecast_init = '2019-12-25'

global training_size_shape
training_size_shape = np.array((9185,48,96))

global validation_testing_size_shape
validation_testing_size_shape = np.array((1144,48,96))

'''This decides how many lag weeks we have for data such as pwat, z200, tmax, etc'''
global observation_lag_list_not_RZSM
#This is for observations pwat, z200, spfh, tmax, diff_temp variables used as predictors
#These are the day lags which were already computed as the 7-day rolling mean

observation_lag_list_not_RZSM = [-1,-7,-14] 



In [7]:
#For testing
# lead=1
# num_lags_obs_RZSM = 6
# include_lags_obs_pwat_spfh_tmax = True
# include_reforecast_or_not=True
# deep_supervision = True
# initial_learning_rate = 0.0001
# beta_1 = 0.9
# batch_size=66
# epochs=1
# shuffle=False
# kernel_norm =  None
# patience=10

In [8]:
def load_all_data_EX0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                      observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                      validation_testing_size_shape,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,final_testing_year):

    soil_var = 'soilw_bgrnd' #This is the observation verification variable name
    
    channel_list = []

    if addtl_experiment == False:
        assert lead >=1, 'Lead must be >=1, do not look at Week 0'
        
        lead_list = np.arange(1,lead+1)

        print(f'Only adding Reforecast data as input for leads {list(lead_list)}.')

        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],lead)) #We are adding both RZSM and/or Tmax (so use * 2)
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead))

        #Add reforecast data
        for idx,lead in enumerate(lead_list):
            channel_list.append(f'RZSM_ref_lead{lead}')

            training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx,ref_source, final_testing_year)

        print(f'Index idx value is {idx}. Done adding RZSM obs.')    

    
    elif addtl_experiment == True:
        print(f'Only adding Reforecast data as input for lead {lead}.')

        lead_add = 1 #We are only adding the current week of reforecast
        lead_list = [lead]

        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],lead_add)) #We are adding both RZSM and Tmax (so use * 2)
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead_add))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead_add))

        channel_list.append(f'RZSM_ref_lead{lead}')

        idx = 0
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx,ref_source, final_testing_year)

    return(training_input, validation_input, testing_input, channel_list)

In [9]:
def load_all_data_EX1_EX2_EX3_EX4_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                           observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                           validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,
                                               experiment_test, experiment_name_out,final_testing_year):
    channel_list = []
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

    assert lead >=1, 'Lead must be >=1, we are not testing week 0 anymore'

    if addtl_experiment == False:
        #Get the number of extra channels to add from previous weeks predictions
        if RZSM_or_Tmax_or_both == 'both':
            add_channels = lead * 2
        else:
            #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
            if lead > 1:
                add_channels = lead - 1
            else:
                add_channels = 0



        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))

        print(f'Training shape input is {training_input.shape}')

        #Add RZSM observations first
        for idx,lag in enumerate(lag_integer_list):
            channel_list.append(f'RZSM_obs_lag{lag}')
            training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)

        print(f'Index idx value is {idx}. Done adding RZSM obs.')
        
        RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)

        
        for variable in var_list:
            channel_name_output = pred.return_channel_name(variable)
            for lag in observation_lag_list_not_RZSM:
                channel_list.append(f'{channel_name_output}_obs_lag{lag}')
                idx+=1
                #Observations adding
                training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx, final_testing_year)
     
        
            print(f'Index idx value is {idx}. Done adding {variable} obs.')
    
        if lead in [0,1]:
            pass
        else:
            #Now we need to load our data from the previous weeks
            for lead_previous in range(1,lead):
                print(f'Working on previous lead: Week {lead_previous}')
                # break
    
                pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
                testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
    
                os.system(f'mkdir -p {pred_dir}')
    
                training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
                validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
                testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
    
                testing_prediction = np.load(testing_prediction_name)
                
                '''Load the training and validation predictions if they exist'''
                if os.path.exists(training_prediction_name):
                    print(f'Loading data from previous week lead {lead_previous}')
                    training_prediction = np.load(training_prediction_name)
                    validation_prediction =np.load(validation_prediction_name)
    
                else:
                    print(f'Creating prediction data from previous week lead {lead_previous}')
                    '''If they don't exist..... load model for previous week'''
                    model = load_model(f'checkpoints/Wk{lead_previous}/Wk{lead_previous}_{experiment_name_out}',compile=False) #don't need the custom loss function for predictions
    
                    training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
                    validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
                    testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
    
                    '''Make predictions and save data to load for previous use'''
                    training_prediction = model.predict(training_input_previous) #Just for unit checking
                    np.save(training_prediction_name,training_prediction)
    
                    validation_prediction = model.predict(validation_input_previous)
                    np.save(validation_prediction_name,validation_prediction)
    
                ############# ADD BACK TO CREATED ARRAY ###############################
                '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
                min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,experiment_name_out,RZSM_or_Tmax_or_both,region_name)
    
                if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
                    min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
                elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
                    min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
                '''Now split the training and validation data back into RZSM and/or Tmax'''
    
    
                train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
                val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
                test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]

    
                '''Make sure we mask RZSM values which aren't on land'''
                train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
                val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
                test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)
    
                #Now add back to the data
                '''Now add back to the newly created dataset'''
                idx+=1
                channel_list.append(f'RZSM_prediction_lead{lead_previous}')
                print(f'Adding RZSM training, validation, testing into index {idx}')
                training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
                validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
                testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]
    
            if RZSM_or_Tmax_or_both == 'both':
                channel_list.append(f'tmax_prediction_lead{lead_previous}')
                idx+=1
                print(f'Adding Tmax training, validation, testing into index {idx}')
                training_input[:,:,:,idx] = train_tmax[:,:,:,0]
                validation_input[:,:,:,idx] = val_tmax[:,:,:,0]
                testing_input[:,:,:,idx] = test_tmax[:,:,:,0]
    

            print('Final channel of input is the following (just make sure it has zeros for RZSM)')
            print(training_input[0,:,:,-1])

    elif addtl_experiment == True:
        #Get the number of extra channels to add from previous weeks predictions
        add_channels = 0


        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))


        #Add RZSM observations first
        for idx,lag in enumerate(lag_integer_list):
            channel_list.append(f'RZSM_obs_lag{lag}')
            training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)

        print(f'Index idx value is {idx}. Done adding RZSM obs.')    
        
        RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)


        for variable in var_list:
            channel_name_output = pred.return_channel_name(variable)
            for lag in observation_lag_list_not_RZSM:
                channel_list.append(f'{channel_name_output}_obs_lag{lag}')
                idx+=1
                #Observations adding
                training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx, final_testing_year)
     
        
            print(f'Index idx value is {idx}. Done adding {variable} obs.')

        print('Final channel of input is the following (just make sure it has zeros for RZSM)')
        print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)

In [10]:
def load_all_data_EX5_EX6_EX7_EX8_experiments(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                              include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list,
                                              input_directory,training_size_shape,validation_testing_size_shape,
                                              RZSM_or_Tmax_or_both,experiment_test,experiment_name,experiment_name_out,final_testing_year):
    print('\nUsing observations as predictions and forecast lead week 1\n')
    #Observations RZSM
    '''We need to combine all the RZSM files into 1 single dictionary for later processing'''
    
    channel_list = []

    assert lead >=1, 'We are only looking at weekly leads 1-5, cannot do 0 lead'
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']
    
    print(f'Experiment name is {experiment_name_out}')
    
    if experiment_name in ['EX22','EX23','EX24','EX25']:
        #This is only a bias correction using observations (no reforecast input at all)
        lead_add = 0
    elif experiment_name in ['EX5','EX6','EX7','EX8']:
        #This is only a bias correction using observations and the current week of reforecast as inputs
        lead_add = 1
    elif experiment_name in ['EX18','EX19','EX20','EX21']:
        if lead ==1:
            lead_add = 1 #Only adding the current week of forecast
        else:
            lead_add = lead #Adding previous weeks forecast and the current week

    print(f'Adding {lead_add} to channel list from reforecast.')

        
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+ lead_add))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list) + lead_add))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+ lead_add))
    
    print(f'Training shape: {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)

    print(f'Index idx value is {idx}. Done adding RZSM obs.')
    
    #For masking
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)

    if experiment_name in ['EX5','EX6','EX7','EX8']:
        #We only need to add the current reforecast week
        #Add final predictor for the current week of RZSM within reforecast
        idx+=1
        soil_var = 'soilw_bgrnd'
        channel_name_output = pred.return_channel_name(soil_var)
        channel_list.append(f'{channel_name_output}_ref_lead{lead}')
    
        print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
        
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx, ref_source,final_testing_year)
        
        return(training_input, validation_input, testing_input, channel_list)

    elif experiment_name in ['EX22','EX23','EX24','EX25']:
         #We are not adding any additional predictors
         return(training_input, validation_input, testing_input, channel_list)
        
    
    else:
        assert lead >= 2, 'If trying to run EX18-EX21, DO NOT RUN with week 1 data. Start with Week 2'



        #Now we need to load our data from the previous weeks
        for lead_previous in range(1,lead):

            #To not have to re-run a bunch more models, we can re-use the training from a previous week
            if lead_previous == 1:
                if experiment_name == 'EX18':
                    prev_experiment = f'EX5_{save_loss_name}_RZSM'
                        
                elif experiment_name == 'EX19':
                    prev_experiment = f'EX6_{save_loss_name}_RZSM'    
                    
                elif experiment_name == 'EX20':
                    prev_experiment = f'EX7_{save_loss_name}_RZSM'  
                    
                elif experiment_name == 'EX21':
                    prev_experiment = f'EX8_{save_loss_name}_RZSM'
            elif lead > 2:
                prev_experiment = experiment_name_out
            
            print(f'Working on previous lead: Week {lead_previous}')
            # break

            pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'

            os.system(f'mkdir -p {pred_dir}')

            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{prev_experiment}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{prev_experiment}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{prev_experiment}.npy'


            testing_prediction = np.load(testing_prediction_name)
            '''Load the training and validation predictions if they exist'''
            
            if os.path.exists(training_prediction_name):
                print(f'Loading data from previous week lead {lead_previous}')
                training_prediction = np.load(training_prediction_name)
                validation_prediction =np.load(validation_prediction_name)

            else:
                print(f'Creating prediction data from previous week lead {lead_previous} from {prev_experiment}.')
                '''If they don't exist..... load model for previous week'''
                model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{prev_experiment}',compile=False) #don't need the custom loss function for predictions

                training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_training_input.npy')
                validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_validation_input.npy')
                testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_testing_input.npy')

                '''Make predictions and save data to load for previous use'''
                training_prediction = model.predict(training_input_previous) #Just for unit checking
                np.save(training_prediction_name,training_prediction)

                validation_prediction = model.predict(validation_input_previous)
                np.save(validation_prediction_name,validation_prediction)

            ############# ADD BACK TO CREATED ARRAY ###############################
            '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
            min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,prev_experiment,RZSM_or_Tmax_or_both, region_name)

            if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
            elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
            '''Now split the training and validation data back into RZSM and/or Tmax'''


            train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
            val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
            test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]


            '''Make sure we mask RZSM values which aren't on land'''
            train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
            val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
            test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)

            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
            validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
            testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]


        #Add final predictor for the current week of RZSM within reforecast
        idx+=1
        soil_var = 'soilw_bgrnd'
        channel_name_output = pred.return_channel_name(soil_var)
        channel_list.append(f'{channel_name_output}_ref_lead{lead}')
    
        print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
        
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx, ref_source,final_testing_year)
    
        print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

        print('Final channel of input is the following (just make sure it has zeros for RZSM)')
        print(training_input[0,:,:,-1])


        return(training_input, validation_input, testing_input, channel_list)

In [11]:
def load_all_data_EX9_EX10_EX11_EX12_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                           observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                           validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both, experiment_test,region_name,experiment_name_out, final_testing_year):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''


    if final_testing_year == None:
        add_year = ''
    else:
        add_year = f'_{final_testing_year}'

    channel_list = []
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

    if experiment_test == 0:
        add_channels = lead  
        reforecast_predictors = False
    elif (experiment_test >= 1) and (lead in [1,2]):
        #This is adding 5 additional predictors (tmax, diff_temp, z200, pwat, spfh)
        #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
        add_channels = lead  + len(var_list)
        reforecast_predictors = True
    elif (experiment_test >= 1) and (lead > 2):
        add_channels = lead 
        reforecast_predictors = False
            
   
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)
        
    print(f'Index idx value is {idx}. Done adding RZSM obs.')    
    
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)

    for variable in var_list:
        channel_name_output = pred.return_channel_name(variable)
        for lag in observation_lag_list_not_RZSM:
            channel_list.append(f'{channel_name_output}_obs_lag{lag}')
            idx+=1
            #Observations adding
            training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx, final_testing_year)
 
    
        print(f'Index idx value is {idx}. Done adding {variable} obs.')
    
    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
    #Now we need to load our data from the previous weeks 
    
    if lead in [0, 1]:
        #DO NOT INCLUDE LEAD 0 AS A PREDICTOR (FROM PREVIOUS WEEK)
        pass
    else:

        for lead_previous in range(1,lead):
            print(f'Adding previous RZSM prediction from week {lead_previous} as an input channel')
            # break
            
            pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
            
            os.system(f'mkdir -p {pred_dir}')
            
            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
            
            testing_prediction = np.load(testing_prediction_name)
            
            '''Load the training and validation predictions if they exist'''
            if os.path.exists(training_prediction_name):
                print(f'Loading data from previous week lead {lead_previous}')
                training_prediction = np.load(training_prediction_name)
                validation_prediction =np.load(validation_prediction_name)
                
            else:
                print(f'Creating prediction data from previous week lead {lead_previous}')
                '''If they don't exist..... load model for previous week'''
                model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{experiment_name_out}',compile=False) #don't need the custom loss function for predictions
                
                training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
                validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
                testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
                
                '''Make predictions and save data to load for previous use'''
                training_prediction = model.predict(training_input_previous) #Just for unit checking
                np.save(training_prediction_name,training_prediction)
                
                validation_prediction = model.predict(validation_input_previous)
                np.save(validation_prediction_name,validation_prediction)
            
            
            ############# ADD BACK TO CREATED ARRAY ###############################
            '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
            min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,experiment_name_out,RZSM_or_Tmax_or_both,region_name)
    
            if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
            elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
            '''Now split the training and validation data back into RZSM and/or Tmax'''
            
            
            train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
            val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
            test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]

    
            '''Make sure we mask RZSM values which aren't on land'''
            train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
            val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
            test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)
    
            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
            validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
            testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]

    
    ############################### NOW ADD THE REFORECAST DATA FROM THE WEEK TO BE PREDICTED ################################################
    
    #Add predictors from the reforecast model
    if (lead in [1,2]) and (experiment_test == 1):
        print(f'Adding current reforecast data from week {lead} for vars {(var_list)}')  
        #Only add these for week 1 and 2. 
        for variable in var_list:
            idx+=1
            channel_name_output = pred.return_channel_name(variable)
            
            channel_list.append(f'{channel_name_output}_ref_lead{lead}')
            
            training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory, variable, idx, ref_source, final_testing_year)
    
            print(f'Index idx value is {idx}. Done adding {variable} reforecast.')

    #Add final predictor for the current week of RZSM within reforecast
    idx+=1
    soil_var = 'soilw_bgrnd'
    channel_name_output = pred.return_channel_name(soil_var)
    channel_list.append(f'{channel_name_output}_ref_lead{lead}')

    print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
    
    training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx,ref_source, final_testing_year)

    print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

    print('Final channel of input is the following (just make sure it has zeros for RZSM)')
    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [12]:
def load_all_data_EX29_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                           observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                           validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both, experiment_test,region_name,experiment_name_out):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''

    channel_list = []
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

    if (experiment_test == 2) and (lead ==1):
        #This is adding 5 additional predictors (tmax, diff_temp, z200, pwat, spfh)
        #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
        add_channels = len(lag_integer_list)+len(var_list)*2  #Only add a single prediction week from these
        reforecast_predictors = True
    elif (experiment_test == 2) and (lead ==2):
        add_channels = len(lag_integer_list) + len(var_list)*2 + (lead -1) 
        reforecast_predictors = True   
    elif (experiment_test == 2) and (lead > 2):
        add_channels = len(lag_integer_list) + (lead -1) 
        reforecast_predictors = False  
        
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)
        
    print(f'Index idx value is {idx}. Done adding RZSM obs.')    
    
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)

    if lead <=2:
        for variable in var_list:
            channel_name_output = pred.return_channel_name(variable)
            for lag in [-1]:
                '''Only include the first lag'''
                channel_list.append(f'{channel_name_output}_obs_lag{lag}')
                idx+=1
                #Observations adding
                training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx,final_testing_year)
     
        
            print(f'Index idx value is {idx}. Done adding {variable} obs.')
        
    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
    #Now we need to load our data from the previous weeks 
    
    if lead in [0, 1]:
        #DO NOT INCLUDE LEAD 0 AS A PREDICTOR (FROM PREVIOUS WEEK)
        pass
    else:

        for lead_previous in range(1,lead):
            print(f'Adding previous RZSM prediction from week {lead_previous} as an input channel')
            # break
            
            pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
            
            os.system(f'mkdir -p {pred_dir}')
            
            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
            
            testing_prediction = np.load(testing_prediction_name)
            
            '''Load the training and validation predictions if they exist'''
            if os.path.exists(training_prediction_name):
                print(f'Loading data from previous week lead {lead_previous}')
                training_prediction = np.load(training_prediction_name)
                validation_prediction =np.load(validation_prediction_name)
                
            else:
                print(f'Creating prediction data from previous week lead {lead_previous}')
                '''If they don't exist..... load model for previous week'''
                model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{experiment_name_out}',compile=False) #don't need the custom loss function for predictions
                
                training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
                validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
                testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
                
                '''Make predictions and save data to load for previous use'''
                training_prediction = model.predict(training_input_previous) #Just for unit checking
                np.save(training_prediction_name,training_prediction)
                
                validation_prediction = model.predict(validation_input_previous)
                np.save(validation_prediction_name,validation_prediction)
            
            
            ############# ADD BACK TO CREATED ARRAY ###############################
            '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
            min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,experiment_name_out,RZSM_or_Tmax_or_both,region_name)
    
            if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
            elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
            '''Now split the training and validation data back into RZSM and/or Tmax'''
            
            
            train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
            val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
            test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]

    
            '''Make sure we mask RZSM values which aren't on land'''
            train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
            val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
            test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)
    
            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
            validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
            testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]

    
    ############################### NOW ADD THE REFORECAST DATA FROM THE WEEK TO BE PREDICTED ################################################
    
    #Add predictors from the reforecast model
    if (lead in [1,2]):
        print(f'Adding current reforecast data from week {lead} for vars {(var_list)}')  
        #Only add these for week 1 and 2. 
        for variable in var_list:
            idx+=1
            channel_name_output = pred.return_channel_name(variable)
            
            channel_list.append(f'{channel_name_output}_ref_lead{lead}')
            
            training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory, variable, idx, ref_source, final_testing_year)
    
            print(f'Index idx value is {idx}. Done adding {variable} reforecast.')

    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [13]:
def load_only_predictions_EX26(lead, input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_name,region_name,
                               experiment_name_out,final_testing_year):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''

    channel_list = []

    assert lead >=2, 'We cannot make a prediction if lead week is 1 because we dont have an input. So make lead week >= 2.'

    #Get the number of extra channels to add from previous weeks predictions
    add_channels = lead # We are only adding the previous weeks lag and the current week. 

    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #For masking
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)

    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
        #Now we need to load our data from the previous weeks
    for idx,lead_previous in enumerate(range(1,lead)):
        # break
        
        pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
        testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
        
        os.system(f'mkdir -p {pred_dir}')
        
        previous_model = return_fully_autoregressive_EX26(lead)
            
        
        training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{previous_model[idx]}.npy'
        validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{previous_model[idx]}.npy'
        testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{previous_model[idx]}.npy'
        
        testing_prediction = np.load(testing_prediction_name)
        '''Load the training and validation predictions if they exist'''
        
        if os.path.exists(training_prediction_name):
            print(f'Loading data from previous week lead {lead_previous}')
            training_prediction = np.load(training_prediction_name)
            validation_prediction =np.load(validation_prediction_name)
            
        else:
            print(f'Creating prediction data from previous week lead {lead_previous}')
            '''If they don't exist..... load model for previous week'''
            model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{experiment_name}',compile=False) #don't need the custom loss function for predictions
            
            training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
            validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
            testing_input_previous = np.load(f'Data/model_npy_inputs{region_name}//Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
            
            '''Make predictions and save data to load for previous use'''
            training_prediction = model.predict(training_input_previous) #Just for unit checking
            np.save(training_prediction_name,training_prediction)
            
            validation_prediction = model.predict(validation_input_previous)
            np.save(validation_prediction_name,validation_prediction)
        
        
        ############# ADD BACK TO CREATED ARRAY ###############################
        '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
        min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,previous_model[idx],RZSM_or_Tmax_or_both, region_name)

        if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
            min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
        elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
            min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
        '''Now split the training and validation data back into RZSM and/or Tmax'''
        
        
        train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
        val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
        test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]


        '''Make sure we mask RZSM values which aren't on land'''
        train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
        val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
        test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)

        #Now add back to the data
        '''Now add back to the newly created dataset'''
        channel_list.append(f'RZSM_prediction_lead{lead_previous}')
        print(f'Adding RZSM training, validation, testing into index {idx}')
        training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
        validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
        testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]

        idx+=1

    
    ############################### NOW ADD THE REFORECAST DATA FROM THE WEEK TO BE PREDICTED ################################################
    
    #Add RZSM reforecast week N
    soil_var = 'soilw_bgrnd'
    channel_name_output = pred.return_channel_name(soil_var)
    channel_list.append(f'{channel_name_output}_ref_lead{lead}')

    print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
    
    training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx, ref_source,final_testing_year)

    print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

    print('Final channel of input is the following (just make sure it has zeros for RZSM)')
    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [14]:
def make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                             observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                             validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,region_name,experiment_name_out):
    
    
    #Set restrictions. For EX0, we only want to bias correct the forecasts with week lead 1 RZSM and tmax
    if experiment_name == 'EX0' or experiment_name == 'EX13':
        
        #Step 1 load data 
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        final_input_train, final_input_validation, final_input_testing,channel_list =  loadData.load_all_data_EX0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                     observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                     validation_testing_size_shape,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,final_testing_year=final_testing_year)

        print(f'\nInput channels will be only week 1 Tmax and RZSM through the current lead {lead} which are also going to be predicted. There are no observations in this experiment\n')
        
        print('Done')
        
        
    
    ############################################### EXPERIMENTS 1-4 ####################################################################################
    elif experiment_name in ['EX1','EX2','EX3','EX4','EX14','EX15','EX16','EX17']:

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        if lead == 0:
            #Step 1 load data 
            final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX1_EX2_EX3_EX4(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                                                                                                    include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, input_directory,
                                                                                                                   training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,
                                                                                                                                   region_name=region_name)
        else:
            final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX1_EX2_EX3_EX4_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                                                       observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                                                       validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,
                                                                                                                                    experiment_name_out=experiment_name_out,final_testing_year=final_testing_year)

        
                
        print('Done')
        
        
    
    ############################################### EXPERIMENTS 5-8 ####################################################################################
    elif experiment_name in ['EX5','EX6','EX7','EX8','EX18','EX19','EX20','EX21','EX22','EX23','EX24','EX25']:

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        # #Step 1 load data 
        # if addtl_experiment == False:
        #     final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX5_EX6_EX7_EX8(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
        #                                                                                                    include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list,
        #                                                                                                    input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,
                                                                                                                                   # region_name=region_name)
        # else:
        final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX5_EX6_EX7_EX8_experiments(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                      observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                      validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_test,experiment_name,experiment_name_out=experiment_name_out,final_testing_year=final_testing_year)

        print('Done')
        
        
    
    ############################################### EXPERIMENTS 9-12 ####################################################################################
    elif (experiment_name == 'EX9') or (experiment_name == 'EX10') or (experiment_name == 'EX11') or (experiment_name == 'EX12') or (experiment_name == 'EX27') or (experiment_name == 'EX28'):
        
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        if lead == 0:
            final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX9_EX10_EX11_EX12(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                                                                                              include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, 
                                                                                                              input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both, 
                                                                                                                                    experiment_test = experiment_test,
                                                                                                                                   region_name=region_name)
        else:
            final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX9_EX10_EX11_EX12_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                       observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                   validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,
                                                                                                                                       experiment_test = experiment_test,
                                                                                                                                       region_name = region_name,experiment_name_out=experiment_name_out,final_testing_year=final_testing_year)
    elif (experiment_name == 'EX26'):

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        final_input_train, final_input_validation, final_input_testing,
        channel_list = load_only_predictions_EX26(lead, input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_name,region_name,
                               experiment_name_out)
        print('Done')


    ############################################### EXPERIMENTS 29 ####################################################################################
    elif (experiment_name == 'EX29'):
        
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX29_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                   observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                               validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,experiment_test = experiment_test,
                                                                                                                                       region_name = region_name,experiment_name_out=experiment_name_out)
                                                                                                        

    return (final_input_train,final_input_validation,final_input_testing,channel_list)


In [15]:
#To try and save on memory, only load certain files at a time
def return_only_train_validation(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                 observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                 validation_testing_size_shape,experiment_name):
    
    reforecast_train_input, reforecast_validation_input, reforecast_testing_input,channel_list \
    = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name,region_name)

    return(reforecast_train_input, reforecast_validation_input,reforecast_testing_input,channel_list)





def return_testing_data_model_training(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name):

    reforecast_train_input, reforecast_validation_input, reforecast_testing_input,channel_list = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name,region_name)
    return(reforecast_testing_input,channel_list)

# Load Data files

In [16]:
# ## Testing
# region_name=region_name
# lead = lead_week
# num_lags_obs_RZSM=3
# include_lags_obs_pwat_spfh_tmax=True
# include_reforecast_or_not=True
# addtl_experiment = False
# experiment_test = 1

In [17]:
def return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name):

    if final_testing_year == None:
        add_year = ''
    else:
        add_year = f'_{final_testing_year}'
    
    experiment_name = CE.return_experiment_name(include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test)

    save_experiment_name = f'{experiment_name}_RZSM'
    if testing_scenario == 'None':
        experiment_name_out = f'{experiment_name}_RZSM{add_year}'
    else:
        experiment_name_out = f'{experiment_name}_{save_loss_name}_RZSM{add_year}' #Need to manually change this if using different loss functions to keep track of things
    
    global lag_integer_list
    lag_integer_list =  CE.return_num_day_lags_from_weekly_lags(num_lags_obs_RZSM) #For number of RZSM observation lags

    #Where to save channel information
    save_experiment_dir = f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data'
    
    channel_save_dir = f'channel_list_information/Wk{lead}'
    
    checkpoint_filepath = f'checkpoints/{region_name}/Wk{lead}/Wk{lead}_{experiment_name_out}'
    
    losses_dir = f'Losses_with_OBS/{region_name}/Wk{lead}'
    
    save_checkpoint_dir =f'checkpoints/{region_name}/Wk{lead}'
    
    os.system(f'mkdir -p {checkpoint_filepath} {channel_save_dir} {save_experiment_dir} {losses_dir} {save_checkpoint_dir}')

    
    #Set up files for either saving or loading
    training_input_file = f'{save_experiment_dir}/{save_experiment_name}_training_input{add_year}.npy'
    validation_input_file = f'{save_experiment_dir}/{save_experiment_name}_validation_input{add_year}.npy'
    testing_input_file = f'{save_experiment_dir}/{save_experiment_name}_testing_input{add_year}.npy'

    
    return(save_experiment_name,experiment_name_out,experiment_name,lag_integer_list,channel_save_dir,
           training_input_file,validation_input_file,testing_input_file,checkpoint_filepath,
          losses_dir,save_checkpoint_dir,save_experiment_dir)

In [18]:
def generate_model_inputs(region_name,lead,include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test):

    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    if os.path.exists(training_input_file) and os.path.exists(validation_input_file) and os.path.exists(testing_input_file):
        pass
    else:
        reforecast_train_input, reforecast_validation_input, reforecast_testing_input, channel_list \
        = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                             observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                             validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,region_name,experiment_name_out)
    
        #Save data to file
        np.save(training_input_file,tf.convert_to_tensor(reforecast_train_input,dtype=tf.float32))
        np.save(validation_input_file,tf.convert_to_tensor(reforecast_validation_input,dtype=tf.float32))
        np.save(testing_input_file,tf.convert_to_tensor(reforecast_testing_input,dtype=tf.float32))
        
        image_size = reforecast_train_input.shape[1:]           
        #Save channel list information to txt file
    
        with open(f'{channel_save_dir}/{save_experiment_name}_channel_list.txt', 'w') as file:
            for idx,element in enumerate(channel_list):
                if '-' in element:
                    source_='OBSERVATIONS'
                else:
                    source_='REFORECAST'
    
                file.write(f'Channel_{idx} is from {source_} with lead or lag {str(element)}' + '\n')

    return(f'Completed writing lead {lead} model input data to {save_experiment_dir}.')

In [19]:
def train(epochs, batch_size, lead, initial_learning_rate, beta_1,shuffle,patience, kernel_norm,deep_supervision,
         num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,include_reforecast_or_not,
          number_of_UNET_backbone_max_pool,permutation_test,make_additional_predictions_from_model_for_testing,
         input_directory,training_size_shape,validation_testing_size_shape,addtl_experiment,experiment_test,region_name):
    
#For testing
# lead=1
# num_lags_obs_RZSM = 3
# include_lags_obs_pwat_spfh_tmax = True
# include_reforecast_or_not=True
# deep_supervision = True
# initial_learning_rate = 0.0001
# beta_1 = 0.9
# batch_size=66
# epochs=1
# shuffle=False
# kernel_norm =  None
# patience=10
# permutation_test = False
    
    # print(addtl_experiment)
    #Training data
    #Needed for calculating the loss for individual UNET predictions (each has 4 predictions for each variable)
    def return_training_verification_data(RZSM_or_tmax):
        #We had multiple verification inputs in the past, but now we just reduced to RZSM
        if RZSM_or_tmax == 'RZSM':
            return(np.array(tf.expand_dims(obs_final_train,-1)))

    
    #Validation data
    def return_validation_verification_data(RZSM_or_tmax):
        #We had multiple verification inputs in the past, but now we just reduced to RZSM
        if RZSM_or_tmax == 'RZSM':
            return(np.array(tf.expand_dims(obs_final_validation,-1)))

    
    
    def print_shape(file,name):
        print(f'Shape of {name} is {file.shape}')
        
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    obs_final_train,obs_final_validation,obs_final_testing = f.load_verification_observations_updated(lead,verification_directory)
    

    print('Loading previously created data')
    #Have to do this way because of memory issues
    
    if number_of_UNET_backbone_max_pool == 4:
        if RZSM_or_Tmax_or_both == 'both':
            reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                                {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), 'RZSM_output_3': return_training_verification_data('RZSM'),
                                                                                 'tmax_output_1': return_training_verification_data('tmax'), 'tmax_output_2': return_training_verification_data('tmax'), 'tmax_output_3': return_training_verification_data('tmax')}))
            reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                                {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'), 'RZSM_output_3': return_validation_verification_data('RZSM'),
                                                                                 'tmax_output_1': return_validation_verification_data('tmax'), 'tmax_output_2': return_validation_verification_data('tmax'), 'tmax_output_3': return_validation_verification_data('tmax')}))
            # reforecast_testing_input_tensor = tf.data.Dataset.from_tensor_slices((np.load(testing_input_file),obs_final_testing))
            # reforecast_testing_input = np.load(testing_input_file)
        elif RZSM_or_Tmax_or_both == 'RZSM':
            reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                                {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), 'RZSM_output_3': return_training_verification_data('RZSM')}))
            reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                                {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'), 'RZSM_output_3': return_validation_verification_data('RZSM')}))

        image_size = np.array(list(reforecast_train_input_tensor.element_spec)[0]['input_image'].shape)
        
    elif number_of_UNET_backbone_max_pool == 5:
        if RZSM_or_Tmax_or_both == 'both':
            reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                                    {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), 'RZSM_output_3': return_training_verification_data('RZSM'),'RZSM_output_4': return_training_verification_data('RZSM'),
                                                                                     'tmax_output_1': return_training_verification_data('tmax'), 'tmax_output_2': return_training_verification_data('tmax'), 'tmax_output_3': return_training_verification_data('tmax'),'tmax_output_4': return_training_verification_data('tmax')}))
            reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                                    {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'), 'RZSM_output_3': return_validation_verification_data('RZSM'),'RZSM_output_4': return_validation_verification_data('RZSM'),
                                                                                     'tmax_output_1': return_validation_verification_data('tmax'), 'tmax_output_2': return_validation_verification_data('tmax'), 'tmax_output_3': return_validation_verification_data('tmax'), 'tmax_output_4': return_validation_verification_data('tmax')}))
        elif RZSM_or_Tmax_or_both == 'RZSM':
            reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                                    {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), 'RZSM_output_3': return_training_verification_data('RZSM'),'RZSM_output_4': return_training_verification_data('RZSM')}))
            reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                                    {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'), 'RZSM_output_3': return_validation_verification_data('RZSM'),'RZSM_output_4': return_validation_verification_data('RZSM')}))
            # reforecast_testing_input_tensor = tf.data.Dataset.from_tensor_slices((np.load(testing_input_file),obs_final_testing))
            # reforecast_testing_input = np.load(testing_input_file)

        image_size = np.array(list(reforecast_train_input_tensor.element_spec)[0]['input_image'].shape)

        # reforecast_train_input = np.load(training_input_file)
        # reforecast_validation_input  = np.load(validation_input_file)
        # reforecast_testing_input  = np.load(testing_input_file)
        # image_size = np.array(reforecast_train_input).shape
  
    
    with open(f'{channel_save_dir}/{save_experiment_name}_channel_list.txt', 'r') as file:
        # Read all lines from the file into a list
        channel_list = file.readlines()

    # print_shape(reforecast_train_input,'Training input')
    # print_shape(reforecast_validation_input,'Validation input')
    # print_shape(reforecast_testing_input,'Testing input')
    # print_shape(obs_final_train,'Observation verification training')
    # print_shape(obs_final_validation,'Observation verification validation')
    # print_shape(obs_final_testing,'Observation verification testing')
    

    print('Actual channel order list')
    print(channel_list)

    inputs = Input(shape=image_size, name='input_image')
    

    build_model = UNETRzsm.model_build_func(inputs=inputs, output_channels=1, 
                           using_deep_supervision=deep_supervision, 
                           kernel_norm = kernel_norm , var_name='RZSM',
                          number_of_UNET_backbone_max_pool = number_of_UNET_backbone_max_pool)
    model = Model(inputs=inputs,
             outputs = build_model,
             name="UNET_RZSM")


    
    # print(model.summary())

    if number_of_UNET_backbone_max_pool == 4:
        if RZSM_or_Tmax_or_both =='both':
            model.compile(loss= {'tmax_output_1':loss_fn, 'tmax_output_2':loss_fn, 'tmax_output_3':loss_fn, 
                                 'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn, 'RZSM_output_3':loss_fn},
                          metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)
        elif RZSM_or_Tmax_or_both =='RZSM':
            model.compile(loss= {'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn, 'RZSM_output_3':loss_fn},
                      metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)
            
    elif number_of_UNET_backbone_max_pool == 5:
        if RZSM_or_Tmax_or_both =='both':
        # With all return channels
            model.compile(loss= {'tmax_output_1':loss_fn, 'tmax_output_2':loss_fn, 'tmax_output_3':loss_fn, 'tmax_output_4':loss_fn,
                                 'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn, 'RZSM_output_3':loss_fn, 'RZSM_output_4':loss_fn},
                metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)
        elif RZSM_or_Tmax_or_both =='RZSM':
            model.compile(loss= {'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn, 'RZSM_output_3':loss_fn, 'RZSM_output_4':loss_fn},
                metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)


    '''For some reason I can't save anymore to my home space, so let's just move it to scratch'''

    checkpoint_filepath_out = f'/glade/derecho/scratch/klesinger/{checkpoint_filepath}'
    save_checkpoint_dir_out = f'/glade/derecho/scratch/klesinger/{save_checkpoint_dir}'

    os.system(f'rm {checkpoint_filepath} -r') #must do this so that we can make a soft link
    os.system(f'mkdir -p {save_checkpoint_dir_out}')
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_out,
    save_weights_only=True,
    mode='min',
    save_best_only=True)
    
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    
    my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience),model_checkpoint_callback, rlrop] 
    
    unet_history=model.fit(
        x=reforecast_train_input_tensor.batch(batch_size), 
        batch_size=batch_size, epochs=epochs, initial_epoch=0, shuffle=shuffle, callbacks=my_callbacks, \
        validation_data=reforecast_validation_input_tensor.batch(batch_size))

    loss_out_df = pd.DataFrame(unet_history.history)
    loss_out_df.to_csv(f'{losses_dir}/Wk{lead}_{experiment_name_out}')


    ########### MODEL CHECKPOINTS ############################

    model.save(f'{save_checkpoint_dir_out}/Wk{lead}_{experiment_name_out}')
    os.system(f'ln -s {save_checkpoint_dir_out}/Wk{lead}_{experiment_name_out} /glade/work/klesinger/FD_RZSM_deep_learning/{save_checkpoint_dir}/Wk{lead}_{experiment_name_out}')
              
    return(0)


    

In [20]:
# # #Testing for model runs
# permutation_test=False
# lead=1
# num_lags_obs_RZSM =3
# include_lags_obs_pwat_spfh_tmax = True
# include_reforecast_or_not=True
# deep_supervision = True
# initial_learning_rate = 0.0001
# beta_1 = 0.9
# batch_size=66
# epochs=1
# shuffle=False
# kernel_norm =  None
# patience=10
# number_of_UNET_backbone_max_pool=4
# make_additional_predictions_from_model_for_testing=False
# addtl_experiment = False

# Run the model for each experiment

In [21]:
'''For experiments, you need to run the script once to make the data. Then restart the kernel to load the data. 
Otherwise the memory leaks are too high'''
def run_EXPERIMENT(lead, num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax ,include_reforecast_or_not,addtl_experiment,experiment_test,region_name):
    if lead_week == 5:
        patience=15
    else:
        patience = 10

    #Options for num_lags_obs_RZSM weekly lags [ 3,6,9,12 ] ** this inc
    #Options for num_lags_obs_pwat_spfh_tmax weekly lags [0 weeks or 2 weeks ]

    train(epochs = 300, 
          batch_size = 66, 
          lead=lead_week, 
          initial_learning_rate=0.0001,
          beta_1 = 0.9,
          shuffle=False,
          patience=patience,
          kernel_norm = None,
          deep_supervision = True,
          num_lags_obs_RZSM=num_lags_obs_RZSM, 
          include_lags_obs_pwat_spfh_tmax=include_lags_obs_pwat_spfh_tmax,
          include_reforecast_or_not=include_reforecast_or_not,
          number_of_UNET_backbone_max_pool=4,
          permutation_test = False,
          make_additional_predictions_from_model_for_testing = False,
          input_directory=input_directory,
          training_size_shape=training_size_shape,
          validation_testing_size_shape=validation_testing_size_shape,
          addtl_experiment = addtl_experiment,
          experiment_test =experiment_test,
          region_name = region_name)
    return(0)



In [22]:
def make_single_predictions(lead, include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test, region_name):
    
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    ########### MAKE PREDICTIONS ON TRAINING, TESTING, VALIDATION SET ############################
    # model = load_model(f'checkpoints/Wk_{lead}/Wk{lead}_{experiment_name_out}',compile=False)

    print(f'Starting single prediction for Experiment {experiment_name_out}')
       
    model = load_model(checkpoint_filepath,compile=False) #don't need the custom loss function for predictions
    
    #Load data
    reforecast_training_input = np.load(training_input_file)
    reforecast_validation_input = np.load(validation_input_file)
    reforecast_testing_input = np.load(testing_input_file)
    
    ########### SAVE PREDICTIONS (SINGLE PREDICTIONS) ############################
    train_val_dir = f'predictions/{region_name}/Wk{lead}_training_validation'
    test_dir = f'predictions/{region_name}/Wk{lead}_testing'

    os.system(f'mkdir -p {test_dir} {train_val_dir}')

    mask = masks.load_mask(region_name)

    
    print('\nCurrently only predicting the test dataset\n')
    predictions = model.predict(reforecast_testing_input)
    # predictions.shape
    np.save(f'{test_dir}/Wk{lead}_testing_{experiment_name_out}.npy',predictions)

    print('\nPredicting training and validation input')
    predictions = model.predict(reforecast_training_input)
    # predictions.shape
    np.save(f'{train_val_dir}/Wk{lead}_training_{experiment_name_out}.npy',predictions)
    
    predictions = model.predict(reforecast_validation_input)
    # predictions.shape
    np.save(f'{train_val_dir}/Wk{lead}_validation_{experiment_name_out}.npy',predictions)
    
    return(f'Completed Experiment {experiment_name_out}')

In [23]:
#These are models which only include the reforecast predictions. We are not including any observations into them.

EX0 ={'region_name':region_name, 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX13={'region_name':region_name, 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }


In [24]:
#Include all predictors for obs but DO NOT include the current reforecast RZSM 

EX1 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX2 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX3 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX4 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }


#Include all predictors for obs but DO include the current reforecast RZSM 
EX14 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':1 }
EX15 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':2 }
EX16 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':3 }
EX17 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':4 }


In [25]:
#Include only OBS RZSM predictors and the current reforecast RZSM week 

EX5 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX6 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX7 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX8 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }

#Include only OBS RZSM predictors, previous weeks prediction, and current reforecast RZSM week 
EX18 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':1 }
EX19 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':2 }
EX20 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':3 }
EX21 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':4 }

#Include only OBS RZSM predictors DO NOT INCLUDE ANY REFORECAST OR PREDICTION
EX22 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':5 }
EX23 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':6 }
EX24 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':7 }
EX25 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':8 }


In [26]:
#Include only OBS RZSM predictors, OBS other variables, previous predictions of RZSM, and current reforecast lead

EX9 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX10 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX11 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX12 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }


In [27]:

# EX26 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }

# run_EXPERIMENT(num_lags_obs_RZSM=0,include_lags_obs_pwat_spfh_tmax=False,include_reforecast_or_not=True, addtl_experiment = False, experiment_test = 26)

In [28]:
#This like EX9 and EX10 except now we are adding additional reforecast predictors from week 1 and 2

EX27 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }
EX28 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }

EX29 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':2 }


In [29]:
def run_model_and_prediction(experiment):
    global lead_week
    for lead_week in [1,2,3,4,5]:
        global lead
        lead=lead_week
    
        generate_model_inputs(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])
    
        run_EXPERIMENT(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])
    
        make_single_predictions(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])

In [30]:
run_model_and_prediction(experiment=EX29)
t


Working on setting up data for Experiment EX29 for lead 1

Training input shape = (9185, 48, 96, 13)
Index idx value is 2. Done adding RZSM obs.
Loading observations for pwat_eatm and lag -1
Index idx value is 3. Done adding pwat_eatm obs.
Loading observations for spfh_2m and lag -1
Index idx value is 4. Done adding spfh_2m obs.
Loading observations for tmax_2m and lag -1
Index idx value is 5. Done adding tmax_2m obs.
Loading observations for diff_temp_2m and lag -1
Index idx value is 6. Done adding diff_temp_2m obs.
Loading observations for hgt_pres and lag -1
Index idx value is 7. Done adding hgt_pres obs.
Adding current reforecast data from week 1 for vars ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']
Index idx value is 8. Done adding pwat_eatm reforecast.
Index idx value is 9. Done adding spfh_2m reforecast.
Index idx value is 10. Done adding tmax_2m reforecast.
Index idx value is 11. Done adding diff_temp_2m reforecast.
Index idx value is 12. Done adding hgt_pre

2024-05-21 15:18:18.253753: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [9185,48,96,1]
	 [[{{node Placeholder/_3}}]]
2024-05-21 15:18:18.404661: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8907
2024-05-21 15:18:28.535282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


140/140 [==============================] - ETA: 0s - loss: 0.3065 - RZSM_output_1_loss: 0.1327 - RZSM_output_2_loss: 0.0838 - RZSM_output_3_loss: 0.0900 - RZSM_output_1_mae: 0.1426 - RZSM_output_2_mae: 0.0895 - RZSM_output_3_mae: 0.0952

2024-05-21 15:20:55.023282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [1144,48,96,1]
	 [[{{node Placeholder/_3}}]]


140/140 [==============================] - 169s 814ms/step - loss: 0.3065 - RZSM_output_1_loss: 0.1327 - RZSM_output_2_loss: 0.0838 - RZSM_output_3_loss: 0.0900 - RZSM_output_1_mae: 0.1426 - RZSM_output_2_mae: 0.0895 - RZSM_output_3_mae: 0.0952 - val_loss: 0.5065 - val_RZSM_output_1_loss: 0.1719 - val_RZSM_output_2_loss: 0.1707 - val_RZSM_output_3_loss: 0.1638 - val_RZSM_output_1_mae: 0.1752 - val_RZSM_output_2_mae: 0.1732 - val_RZSM_output_3_mae: 0.1665 - lr: 1.0000e-04
Epoch 2/300
140/140 [==============================] - 100s 712ms/step - loss: 0.1431 - RZSM_output_1_loss: 0.0516 - RZSM_output_2_loss: 0.0448 - RZSM_output_3_loss: 0.0467 - RZSM_output_1_mae: 0.0546 - RZSM_output_2_mae: 0.0470 - RZSM_output_3_mae: 0.0492 - val_loss: 0.3023 - val_RZSM_output_1_loss: 0.1006 - val_RZSM_output_2_loss: 0.1027 - val_RZSM_output_3_loss: 0.0990 - val_RZSM_output_1_mae: 0.1037 - val_RZSM_output_2_mae: 0.1051 - val_RZSM_output_3_mae: 0.1016 - lr: 1.0000e-04
Epoch 3/300
140/140 [===============

...

2024-05-21 20:15:38.261089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-05-21 20:15:38.279233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-05-21 20:15:38.297499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-05-21 20

INFO:tensorflow:Assets written to: /glade/derecho/scratch/klesinger/checkpoints/CONUS/Wk5/Wk5_EX29_regular_RZSM_2012/assets


INFO:tensorflow:Assets written to: /glade/derecho/scratch/klesinger/checkpoints/CONUS/Wk5/Wk5_EX29_regular_RZSM_2012/assets


Starting single prediction for Experiment EX29_regular_RZSM_2012

Currently only predicting the test dataset

36/36 [==============================] - 10s 44ms/step

Predicting training and validation input
36/36 [==============================] - 8s 217ms/step


NameError: name 't' is not defined

In [ ]:
experiment=EX27
#for testing
lead,lead_week = 1,1

experiment['region_name']
lead = lead_week
num_lags_obs_RZSM=experiment['num_lags_obs_RZSM']
include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax']
include_reforecast_or_not=experiment['include_reforecast_or_not']
addtl_experiment = experiment['addtl_experiment']
experiment_test = experiment['experiment_test']

epochs = 150
batch_size = 66
initial_learning_rate=0.0001
beta_1 = 0.9
shuffle=False
patience=10
kernel_norm = None
deep_supervision = True
number_of_UNET_backbone_max_pool=4


In [ ]:
    # #EX27
    # generate_model_inputs(region_name=region_name,lead = lead_week,num_lags_obs_RZSM=3,include_lags_obs_pwat_spfh_tmax=True,include_reforecast_or_not=True, addtl_experiment = False, experiment_test = 1)
    # #EX27
    # run_EXPERIMENT(region_name=region_name,lead = lead_week,num_lags_obs_RZSM=3,include_lags_obs_pwat_spfh_tmax=True,include_reforecast_or_not=True, addtl_experiment = False, experiment_test = 1)
    # #EX27
    # make_single_predictions(lead = lead_week,num_lags_obs_RZSM=3,include_lags_obs_pwat_spfh_tmax=True,include_reforecast_or_not=True, addtl_experiment = False, experiment_test = 1,region_name=region_name)

In [ ]:
def return_fully_autoregressive_EX26(lead):

    #We can manually select the best models and create a week N estimate based on them (this implies that all the information from the observations is contained in the first 
    #Set of leads

    best_models = []
    
    if lead == 2:
        previous_model = ['EX10_RZSM','EX20_RZSM']
    elif lead == 3:
        previous_model = ['EX10_RZSM','EX20_RZSM','EX20_RZSM']
    elif lead == 4:
        previous_model = ['EX10_RZSM','EX20_RZSM','EX20_RZSM','EX20_RZSM']
    elif lead == 5:
        previous_model = ['EX10_RZSM','EX20_RZSM','EX20_RZSM','EX20_RZSM','EX20_RZSM']

    return(previous_model)


# Different model experiments

In [ ]:
lead,lead_week = 1,1

In [ ]:


    # if lead_week !=0:

    #     #EX26
    #     run_EXPERIMENT(num_lags_obs_RZSM=0,include_lags_obs_pwat_spfh_tmax=False,include_reforecast_or_not=True, addtl_experiment = False, experiment_test = 26)
        